In [33]:
import pickle
import re
import random
import numpy as np
from gensim.models import word2vec
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [2]:
def readFile(fname):
    f = open(fname, "rb")
    result = pickle.load(f)
    f.close()
    
    return result

In [3]:
def trecPreprocessing(fname):
    with open(fname, 'r') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    content = [w.replace('\t', ' ') for w in content]

    trec = []

    tmp_trec = []
    tmp_dic = {"p": [], "n": []}

    for index in range(len(content)):
        if ("<QApairs id=" in content[index] and index != 1):
            # If questions have no positive or no negative ans, discard it.
            if ((len(tmp_dic["p"]) != 0) and (len(tmp_dic["n"]) != 0)):
                tmp_trec.append(tmp_dic)
                trec.append(tmp_trec)

            tmp_trec = []
            tmp_dic = {"p": [], "n": []}

        elif (content[index] == "<question>"):
            tmp_trec.append(content[index + 1].lower())
        elif (content[index] == "<positive>"):
            tmp_dic["p"].append(content[index + 1].lower())
        elif (content[index] == "<negative>"):
            tmp_dic["n"].append(content[index + 1].lower())
            
    # Last round
    if ((len(tmp_dic["p"]) != 0) and (len(tmp_dic["n"]) != 0)):
        tmp_trec.append(tmp_dic)
        trec.append(tmp_trec)
    
    return trec

In [4]:
def tokenize(text):
    pattern = re.compile('[a-zA-Z]+')
    return (match.group(0) for match in pattern.finditer(text))

def makeValidationFeatuers(trec_val, vec_model, word_index):
    
    q_docs = []
    a_docs = []
    for e in trec_val:
        tmp_q_docs = []
        all_a_docs = []
        
        q = list(tokenize(e[0]))
        
        p = e[1]['p']
        #p = p[random.randrange(len(p))]
        #p = list(tokenize(p))
        
        n = e[1]['n']
        #n = n[random.randrange(len(n))]
        #n = list(tokenize(n))
        
        for token in q:
            if (token in vec_model.wv.vocab):
                
                tmp_q_docs.append(word_index[token])
                
        for sentence in p:
            tmp_a_docs = []
            current_sentence = list(tokenize(sentence))
            for token in current_sentence:
                if (token in vec_model.wv.vocab):
                    tmp_a_docs.append(word_index[token])
            all_a_docs.append(tmp_a_docs)    
            
        for sentence in n:
            tmp_a_docs = []
            current_sentence = list(tokenize(sentence))
            for token in current_sentence:
                if (token in vec_model.wv.vocab):
                    tmp_a_docs.append(word_index[token])
            all_a_docs.append(tmp_a_docs)
            
        q_docs.append(tmp_q_docs)
        a_docs.append(all_a_docs)
        
    return q_docs, a_docs   

"""
def tokenize(text):
    pattern = re.compile('[a-zA-Z]+')
    return (match.group(0) for match in pattern.finditer(text))

def makeValidationFeatuers(trec_val, vec_model, word_index):
    
    q_docs = []
    p_docs = []
    n_docs = []
    for e in trec_val:
        tmp_q_docs = []
        tmp_p_docs = []
        tmp_n_docs = []
        
        q = list(tokenize(e[0]))
        
        p = e[1]['p']
        p = p[random.randrange(len(p))]
        p = list(tokenize(p))
        
        n = e[1]['n']
        n = n[random.randrange(len(n))]
        n = list(tokenize(n))
        
        for token in q:
            if (token in vec_model.wv.vocab):
                
                tmp_q_docs.append(word_index[token])
                
        for token in p:
            if (token in vec_model.wv.vocab):
                tmp_p_docs.append(word_index[token])
                    
        for token in n:
            if (token in vec_model.wv.vocab):
                tmp_n_docs.append(word_index[token])  
                
        q_docs.append(tmp_q_docs)
        p_docs.append(tmp_p_docs)
        n_docs.append(tmp_n_docs)
        
    return q_docs, p_docs, n_docs    
"""

"\ndef tokenize(text):\n    pattern = re.compile('[a-zA-Z]+')\n    return (match.group(0) for match in pattern.finditer(text))\n\ndef makeValidationFeatuers(trec_val, vec_model, word_index):\n    \n    q_docs = []\n    p_docs = []\n    n_docs = []\n    for e in trec_val:\n        tmp_q_docs = []\n        tmp_p_docs = []\n        tmp_n_docs = []\n        \n        q = list(tokenize(e[0]))\n        \n        p = e[1]['p']\n        p = p[random.randrange(len(p))]\n        p = list(tokenize(p))\n        \n        n = e[1]['n']\n        n = n[random.randrange(len(n))]\n        n = list(tokenize(n))\n        \n        for token in q:\n            if (token in vec_model.wv.vocab):\n                \n                tmp_q_docs.append(word_index[token])\n                \n        for token in p:\n            if (token in vec_model.wv.vocab):\n                tmp_p_docs.append(word_index[token])\n                    \n        for token in n:\n            if (token in vec_model.wv.vocab):\n      

In [5]:

def pad(q_docs, a_docs):
    
    a_pad = []
    max_length = 40
    
    q_pad = pad_sequences(q_docs, maxlen=max_length, padding='post')
    for single_doc in a_docs:
        tmp_a_pad = pad_sequences(single_doc, maxlen=max_length, padding='post')
        a_pad.append(tmp_a_pad)
    return q_pad, a_pad


"""
def pad(q_docs, p_docs, n_docs):
    
    max_length = 40
    
    q_pad = pad_sequences(q_docs, maxlen=max_length, padding='post')
    p_pad = pad_sequences(p_docs, maxlen=max_length, padding='post')
    n_pad = pad_sequences(n_docs, maxlen=max_length, padding='post')
    
    return q_pad, p_pad, n_pad
"""

"\ndef pad(q_docs, p_docs, n_docs):\n    \n    max_length = 40\n    \n    q_pad = pad_sequences(q_docs, maxlen=max_length, padding='post')\n    p_pad = pad_sequences(p_docs, maxlen=max_length, padding='post')\n    n_pad = pad_sequences(n_docs, maxlen=max_length, padding='post')\n    \n    return q_pad, p_pad, n_pad\n"

In [6]:
trec_validation = trecPreprocessing("dev-less-than-40.manual-edit.xml")

In [7]:
word_index = readFile("word_index_dic.pkl")
vec_model = word2vec.Word2Vec.load("word2vec.model")

In [37]:
#q_docs, p_docs, n_docs = makeValidationFeatuers(trec_validation, vec_model, word_index)
q_docs, a_docs = makeValidationFeatuers(trec_validation, vec_model, word_index)
#q_pad, p_pad, n_pad = pad(q_docs, p_docs, n_docs)
q_pad, a_pad = pad(q_docs, a_docs)

In [9]:
model = load_model("weights_best_2.hdf5") 

In [41]:
q_pad[0].shape

(40,)

In [16]:
a_pad[0][0]

(40,)

In [42]:
#model.predict([q_pad, p_pad, n_pad])
model.predict([q_pad[0], a_pad[0][0], a_pad[0][0]])

ValueError: Error when checking : expected input_1 to have shape (40,) but got array with shape (1,)